In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## QA Pair 를 생성할 PDF 를 로드합니다.


In [2]:
from unstructured.partition.pdf import partition_pdf


def extract_pdf_elements(filepath):
    """
    PDF 파일에서 이미지, 테이블, 그리고 텍스트 조각을 추출합니다.
    path: 이미지(.jpg)를 저장할 파일 경로
    fname: 파일 이름
    """
    return partition_pdf(
        filename=filepath,
        extract_images_in_pdf=False,  # PDF 내 이미지 추출 활성화
        infer_table_structure=False,  # 테이블 구조 추론 활성화
        chunking_strategy="by_title",  # 제목별로 텍스트 조각화
        max_characters=4000,  # 최대 문자 수
        new_after_n_chars=3800,  # 이 문자 수 이후에 새로운 조각 생성
        combine_text_under_n_chars=2000,  # 이 문자 수 이하의 텍스트는 결합
    )

In [3]:
# PDF 파일 로드
elements = extract_pdf_elements("data/SPRI_AI_Brief_2023년12월호_F.pdf")

In [4]:
# 로드한 TEXT 청크 수
len(elements)

14

In [5]:
print(elements[2])

위험 식별과 완화에 필요한 조치를 포함

주요 7개국(G7)은 미국, 일본, 독일, 영국, 프랑스, 이탈리아, 캐나다를 의미

** 5월 정상회의에는 한국, 호주, 베트남 등을 포함한 8개국이 초청을 받았으나, AI 국제 행동강령에는 우선 G7 국가만 포함하여 채택

n G7은 행동강령을 통해 아래의 조치를 제시했으며, 빠르게 발전하는 기술에 대응할 수 있도록

이해관계자 협의를 통해 필요에 따라 개정할 예정

첨단 AI 시스템의 개발 과정에서 AI 수명주기 전반에 걸쳐 위험을 평가 및 완화하는 조치를 채택하고,

첨단 AI 시스템의 출시와 배포 이후 취약점과 오용 사고, 오용 유형을 파악해 완화

첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투명성을

보장하고 책임성을 강화

산업계, 정부, 시민사회, 학계를 포함해 첨단 AI 시스템을 개발하는 조직 간 정보공유와 사고 발생 시

신고를 위해 협력하고, 위험 기반 접근방식을 토대로 개인정보보호 정책과 위험 완화 조치를 포함하는

AI 거버넌스와 위험 관리 정책을 마련

AI 수명주기 전반에 걸쳐 물리보안, 사이버보안, 내부자 위협 보안을 포함한 강력한 보안 통제 구현

사용자가 AI 생성 콘텐츠를 식별할 수 있도록 워터마크를 비롯하여 기술적으로 가능한 기법으로

신뢰할 수 있는 콘텐츠 인증과 출처 확인 메커니즘을 개발 및 구축

사회적 위험과 안전·보안 문제를 완화하는 연구와 효과적인 완화 대책에 우선 투자하고, 기후 위기

대응, 세계 보건과 교육 등 세계적 난제 해결을 위한 첨단 AI 시스템을 우선 개발

국제 기술 표준의 개발 및 채택을 가속화하고, 개인정보와 지식재산권 보호를 위해 데이터 입력과 수집

시 적절한 보호 장치 구현

☞ 출처: G7, Hiroshima Process International Code of Conduct for Advanced AI Systems, 2023.10.30.

2

1. 정책/법제

2. 기업/산업

3. 기술/연구



## QA Pair 생성


In [6]:
from langchain_core.pydantic_v1 import BaseModel, Field


# 원하는 데이터 구조를 정의합니다.
class QAPair(BaseModel):
    question: str = Field(description="Question generated from the text")
    answer: str = Field(description="Answer related to the question")

In [7]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate only questions based on the below query.
You are an Teacher/Professor in {domain}. 
Your task is to provide exactly **{num_questions}** question(s) for an upcoming quiz/examination. 
You are not to provide more or less than this number of questions. 
The question(s) should be diverse in nature across the document. 
The purpose of question(s) is to test the understanding of the students on the context information provided.
You must also provide the answer to each question. The answer should be based on the context information provided only.

Restrict the question(s) to the context information provided only.
QUESTION and ANSWER should be written in Korean. response in JSON format which contains the `question` and `answer`.
ANSWER should be a complete sentence.

#Format:
```json
{{
    "QUESTION": "바이든 대통령이 서명한 '안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령'의 주요 목적 중 하나는 무엇입니까?",
    "ANSWER": "바이든 대통령이 서명한 행정명령의 주요 목적은 AI의 안전 마련과 보안 기준 마련을 위함입니다."
}},
{{
    "QUESTION": "메타의 라마2가 오픈소스 모델 중에서 어떤 유형의 작업에서 가장 우수한 성능을 발휘했습니까?",
    "ANSWER": "메타의 라마2는 RAG 없는 질문과 답변 및 긴 형식의 텍스트 생성에서 오픈소스 모델 중 가장 우수한 성능을 발휘했습니다."    
}},
{{
    "QUESTION": "IDC 예측에 따르면 2027년까지 생성 AI 플랫폼과 애플리케이션 시장의 매출은 얼마로 전망되나요?",
    "ANSWER": "IDC 예측에 따르면 2027년까지 생성 AI 플랫폼과 애플리케이션 시장의 매출은 283억 달러로 전망됩니다."    
}}
```
"""
)

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=QAPair)

chain = (
    prompt
    | ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    | parser
)  # 체인을 구성합니다.

qa_pair = []

for element in elements[1:]:
    if element.text:
        qa_pair.append(
            chain.invoke(
                {"context": element.text, "domain": "AI", "num_questions": "3"}
            )
        )

```json
{
    "QUESTION": "바이든 대통령이 2023년 10월 30일 발표한 행정명령의 주요 내용 중 하나는 무엇입니까?",
    "ANSWER": "바이든 대통령이 발표한 행정명령의 주요 내용 중 하나는 AI의 안전과 보안 기준 마련입니다."
},
{
    "QUESTION": "G7이 2023년 10월 30일 합의한 '히로시마 AI 프로세스'의 주요 목적은 무엇입니까?",
    "ANSWER": "G7이 합의한 '히로시마 AI 프로세스'의 주요 목적은 AI 기업을 대상으로 AI 위험 식별과 완화를 위한 국제 행동강령을 마련하는 것입니다."
},
{
    "QUESTION": "행정명령에 따르면 AI 시스템의 안전성과 신뢰성을 확인하기 위해 기업이 미국 정부와 공유해야 하는 정보는 무엇입니까?",
    "ANSWER": "행정명령에 따르면 AI 시스템의 안전성과 신뢰성을 확인하기 위해 기업은 안전 테스트 결과와 시스템에 관한 주요 정보를 미국 정부와 공유해야 합니다."
}
``````json
{
    "QUESTION": "G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 무엇입니까?",
    "ANSWER": "G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투명성을 보장하고 책임성을 강화하는 것입니다."
},
{
    "QUESTION": "영국 AI 안전성 정상회의에서 발표된 '블레츨리 선언'의 주요 내용은 무엇입니까?",
    "ANSWER": "'블레츨리 선언'의 주요 내용은 AI 안전 보장을 위해 국가, 국제기구, 기업, 시민사회, 학계를 포함한 모든 이해관계자의 협력이 중요하다는 점을 강조하고, 특히 최첨단 AI 시스템 개발 기업은 안전 평가를 비롯한 적절한 조치를 취하여 AI 시스템의 안전을 보장할 책임이 있다는 것입니다."
},
{
    "QUESTION": "영국 총리 리시 수낙이 AI

In [11]:
qa_pair[8]

[{'QUESTION': 'IDC 설문조사에 따르면 향후 12개월 동안 응답자의 3분의 1이 고려하고 있는 AI 소프트웨어 도입 방식은 무엇인가요?',
  'ANSWER': 'IDC 설문조사에 따르면 향후 12개월 동안 응답자의 3분의 1은 기업이 특정 사용 사례나 응용 영역에서 외부 AI 소프트웨어의 구매를 고려하거나 외부 AI 소프트웨어와 내부 자원의 결합을 고려하고 있습니다.'},
 {'QUESTION': '빌 게이츠는 AI 에이전트가 컴퓨터 사용 방식에 어떤 변화를 가져올 것으로 예상하나요?',
  'ANSWER': '빌 게이츠는 5년 내 일상언어로 모든 작업을 처리할 수 있는 AI 에이전트가 보급되며 컴퓨터를 사용하는 방식이 완전히 바뀔 것으로 예상하고 있습니다.'},
 {'QUESTION': '유튜브는 2024년부터 AI 생성 콘텐츠에 대해 어떤 조치를 의무화할 예정인가요?',
  'ANSWER': '유튜브는 2024년부터 AI 생성 콘텐츠에 AI 라벨 표시를 의무화하기로 했으며, 이를 준수하지 않는 콘텐츠는 삭제하고 크리에이터에 대한 수익 배분도 중단할 수 있다고 설명했습니다.'}]

In [13]:
qa_pair

[{'QUESTION': '바이든 대통령이 2023년 10월 30일 발표한 행정명령의 주요 내용 중 하나는 무엇입니까?',
  'ANSWER': '바이든 대통령이 발표한 행정명령의 주요 내용 중 하나는 AI의 안전과 보안 기준 마련입니다.'},
 {'QUESTION': 'G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 무엇입니까?',
  'ANSWER': 'G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투명성을 보장하고 책임성을 강화하는 것입니다.'},
 {'QUESTION': '한국은 영국 정부와 몇 개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니까?',
  'ANSWER': '한국은 영국 정부와 6개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니다.'},
 {'QUESTION': 'FTC가 생성 AI와 관련하여 주목하고 있는 주요 위험 요소는 무엇입니까?',
  'ANSWER': 'FTC는 생성 AI와 관련하여 소비자의 개인정보 침해, 차별과 편견의 자동화, 사기 범죄 등의 위험에 주목하고 있습니다.'},
 {'QUESTION': '프런티어 모델 포럼이 AI 안전 연구를 위해 조성한 기금의 규모는 얼마입니까?',
  'ANSWER': '프런티어 모델 포럼이 AI 안전 연구를 위해 조성한 기금의 규모는 1,000만 달러입니다.'},
 {'QUESTION': '데이터 출처 탐색기(Data Provenance Explorer) 플랫폼의 주요 기능은 무엇입니까?',
  'ANSWER': '데이터 출처 탐색기 플랫폼의 주요 기능은 데이터셋의 라이선스 상태를 쉽게 파악하고, 주요 데이터셋의 구성과 데이터 계보도를 추적할 수 있게 하는 것입니다.'},
 {'QUESTION': "삼성전자가 공개한 생성 AI 모델 '삼성 가우스'는 어떤 장점을 가지고 있습니까?",
  'ANSWER': '삼성 가우스는 온디바이

In [14]:
# 디버깅을 위한 데이터셋 추가
additional_qa = [
    {
        "QUESTION": "테디노트 유튜브 채널에 대해서 알려주세요.",
        "ANSWER": "테디노트(TeddyNote)는 데이터 분석, 머신러닝, 딥러닝 등의 주제를 다루는 유튜브 채널입니다. 이 채널을 운영하는 이경록님은 데이터 분석과 인공지능에 대한 다양한 강의를 제공하며, 초보자도 쉽게 따라할 수 있도록 친절하게 설명합니다.",
    },
    {
        "QUESTION": "랭체인 관련 튜토리얼은 어디서 찾을 수 있나요?",
        "ANSWER": "테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314",
    },
    {
        "QUESTION": "테디노트 운영자에 대해서 알려주세요",
        "ANSWER": "테디노트(TeddyNote) 운영자는 이경록(Teddy Lee)입니다. 그는 데이터 분석, 머신러닝, 딥러닝 분야에서 활동하는 전문가로, 다양한 교육 및 강의를 통해 지식을 공유하고 있습니다. 이경록님은 여러 기업과 교육기관에서 파이썬, 데이터 분석, 텐서플로우 등 다양한 주제로 강의를 진행해 왔습니다",
    },
]

In [15]:
qa_pair.extend(additional_qa)
qa_pair

[{'QUESTION': '바이든 대통령이 2023년 10월 30일 발표한 행정명령의 주요 내용 중 하나는 무엇입니까?',
  'ANSWER': '바이든 대통령이 발표한 행정명령의 주요 내용 중 하나는 AI의 안전과 보안 기준 마련입니다.'},
 {'QUESTION': 'G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 무엇입니까?',
  'ANSWER': 'G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투명성을 보장하고 책임성을 강화하는 것입니다.'},
 {'QUESTION': '한국은 영국 정부와 몇 개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니까?',
  'ANSWER': '한국은 영국 정부와 6개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니다.'},
 {'QUESTION': 'FTC가 생성 AI와 관련하여 주목하고 있는 주요 위험 요소는 무엇입니까?',
  'ANSWER': 'FTC는 생성 AI와 관련하여 소비자의 개인정보 침해, 차별과 편견의 자동화, 사기 범죄 등의 위험에 주목하고 있습니다.'},
 {'QUESTION': '프런티어 모델 포럼이 AI 안전 연구를 위해 조성한 기금의 규모는 얼마입니까?',
  'ANSWER': '프런티어 모델 포럼이 AI 안전 연구를 위해 조성한 기금의 규모는 1,000만 달러입니다.'},
 {'QUESTION': '데이터 출처 탐색기(Data Provenance Explorer) 플랫폼의 주요 기능은 무엇입니까?',
  'ANSWER': '데이터 출처 탐색기 플랫폼의 주요 기능은 데이터셋의 라이선스 상태를 쉽게 파악하고, 주요 데이터셋의 구성과 데이터 계보도를 추적할 수 있게 하는 것입니다.'},
 {'QUESTION': "삼성전자가 공개한 생성 AI 모델 '삼성 가우스'는 어떤 장점을 가지고 있습니까?",
  'ANSWER': '삼성 가우스는 온디바이

## jsonl 파일로 저장


In [31]:
original_qa

[{'QUESTION': '바이든 대통령이 2023년 10월 30일 발표한 행정명령의 주요 내용 중 하나는 무엇입니까?',
  'ANSWER': '바이든 대통령이 발표한 행정명령의 주요 내용 중 하나는 AI의 안전과 보안 기준 마련입니다.'},
 {'QUESTION': 'G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 무엇입니까?',
  'ANSWER': 'G7 국가들이 채택한 AI 국제 행동강령의 주요 조치 중 하나는 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투명성을 보장하고 책임성을 강화하는 것입니다.'},
 {'QUESTION': '한국은 영국 정부와 몇 개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니까?',
  'ANSWER': '한국은 영국 정부와 6개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했습니다.'},
 {'QUESTION': 'FTC가 생성 AI와 관련하여 주목하고 있는 주요 위험 요소는 무엇입니까?',
  'ANSWER': 'FTC는 생성 AI와 관련하여 소비자의 개인정보 침해, 차별과 편견의 자동화, 사기 범죄 등의 위험에 주목하고 있습니다.'},
 {'QUESTION': '프런티어 모델 포럼이 AI 안전 연구를 위해 조성한 기금의 규모는 얼마입니까?',
  'ANSWER': '프런티어 모델 포럼이 AI 안전 연구를 위해 조성한 기금의 규모는 1,000만 달러입니다.'},
 {'QUESTION': '데이터 출처 탐색기(Data Provenance Explorer) 플랫폼의 주요 기능은 무엇입니까?',
  'ANSWER': '데이터 출처 탐색기 플랫폼의 주요 기능은 데이터셋의 라이선스 상태를 쉽게 파악하고, 주요 데이터셋의 구성과 데이터 계보도를 추적할 수 있게 하는 것입니다.'},
 {'QUESTION': "삼성전자가 공개한 생성 AI 모델 '삼성 가우스'는 어떤 장점을 가지고 있습니까?",
  'ANSWER': '삼성 가우스는 온디바이

In [30]:
with open("data/qa_pair.jsons", "w") as f:
    json.dumps(original_qa)

In [ ]:
import json

with open("qa_pair.jsonl", "w", encoding="utf-8") as f:
    original_qa = json.dumps(qa_pair, ensure_ascii=False)
    
# 디버깅을 위한 데이터셋 추가
additional_qa = [
    {
        "QUESTION": "여러분의 이름을 넣어보세요",
        "ANSWER": "답변을 적어보세요...",
    },
    {
        "QUESTION": "랭체인 관련 튜토리얼은 어디서 찾을 수 있나요?",
        "ANSWER": "테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314",
    },
    {
        "QUESTION": "테디노트 운영자에 대해서 알려주세요",
        "ANSWER": "테디노트(TeddyNote) 운영자는 이경록(Teddy Lee)입니다. 그는 데이터 분석, 머신러닝, 딥러닝 분야에서 활동하는 전문가로, 다양한 교육 및 강의를 통해 지식을 공유하고 있습니다. 이경록님은 여러 기업과 교육기관에서 파이썬, 데이터 분석, 텐서플로우 등 다양한 주제로 강의를 진행해 왔습니다",
    },
]

original_qa = json.loads(original_qa)
original_qa.extend(additional_qa)
original_qa

In [34]:
import json

with open("qa_pair.jsonl", "w", encoding="utf-8") as f:
    for qa in original_qa:
        f.write(json.dumps(qa, ensure_ascii=False) + "\n")

In [16]:
import json

with open("qa_pair.jsonl", "w", encoding="utf-8") as f:
    for qa in qa_pair:
        qa_modified = {
            "instruction": qa["QUESTION"],
            "input": "",
            "output": qa["ANSWER"],
        }
        f.write(json.dumps(qa_modified, ensure_ascii=False) + "\n")

HuggingFace datasets 라이브러리를 사용하여 데이터셋을 로드합니다.


In [ ]:
from datasets import load_dataset

# JSONL 파일 경로
jsonl_file = "qa_pair.jsonl"

# JSONL 파일을 Dataset으로 로드
dataset = load_dataset("json", data_files=jsonl_file)

In [ ]:
dataset

In [ ]:
from huggingface_hub import HfApi

# HfApi 인스턴스 생성
api = HfApi()

# 데이터셋을 업로드할 리포지토리 이름
repo_name = "teddylee777/QA-Dataset-mini"

# 데이터셋을 허브에 푸시
dataset.push_to_hub(repo_name, token="허깅페이스 토큰")